In [277]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages']
sys.prefix = '/home/joaom/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

# Load


In [ ]:

from src import utils
from datetime import datetime
conn = utils.connect_athena(path='../configs/athena.yaml')

In [6]:
df = pd.read_sql_query("""
select 
    *
from spd_sdv_waze_corona.dev_analysis_analysis_daily
""", conn)

In [8]:
metadata = pd.read_sql_query("""
select 
    *
from spd_sdv_waze_corona.analysis_metadata_metadata_ready
""", conn)

In [11]:
df = df.merge(metadata, on='region_slug')

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

In [19]:
stats = df.groupby(['region_slug', 'dow'])['sum_length'].describe().reset_index()

In [23]:
stats = stats.merge(metadata[['region_slug', 'population']], on='region_slug')

In [144]:
df.groupby(['region_slug']).count()['year'].sort_values()

region_slug
santodomingo                    60
belpoman                       123
country_suriname               144
ciudad_guayana                 340
juarez                         343
maracaibo                      348
portauprince                   360
georgetown                     363
tijuana                        363
maracay                        363
queretaro                      364
pueble                         364
portofspain                    364
porto_alergre                  364
medellin                       364
merida                         364
pereira                        364
paramaribo                     364
mendoza                        364
panamacity                     364
natal                          364
nassau                         364
montevideo                     364
quito                          364
aguascalientes                 364
rosario                        364
torreon                        364
toluca                         364
teresina

In [40]:
stats['std_log10'] = stats['std'].apply(np.log10)
stats.pivot_table(columns='dow', values='std_log10', index='region_slug')\
    .iplot(kind='hist', theme='custom', title='Distribution of Standard Deviation per DOW')

In [ ]:
df.iplot()

In [54]:
stats['std_log10'] = stats['std'].apply(np.log10)
stats['population_log10'] = stats['population'].apply(lambda x: np.log10(int(x.split('.')[0].replace(',',''))))
stats.pivot_table(columns='dow', values='std_log10', index='population_log10')\
    .iplot(mode='markers', size=5,
    theme='custom', title='Standard Deviation by Population per DOW',
          xTitle='Population (log10)', yTitle='Standard Deviation (log10 meters)'
          )

In [57]:
stats_c = df.groupby(['region_slug'])['sum_length'].describe().reset_index()
stats_c = stats_c.merge(metadata[['region_slug', 'population']], on='region_slug')

In [62]:
stats_c['std_log10'] = stats_c['std'].apply(np.log10)
stats_c['std_log10'].iplot(kind='hist', theme='custom', title='Distribution of Standard Deviation',
                          xTitle='Standard Deviation (log10 meter)'
                          )

In [71]:
stats_c['std_log10'] = stats_c['std'].apply(np.log10)
stats_c['population_log10'] = stats_c['population'].apply(lambda x: np.log10(int(x.split('.')[0].replace(',',''))))
stats_c.set_index('population_log10')['std_log10'].iplot(
    mode='markers', size=7, theme='custom', title='Standard Deviation by Population',
                          xTitle='Population (log10)', yTitle='Standard Deviation (log10 meter)')

In [ ]:
import plotly.express as px

In [107]:
stats_c['population'] = stats_c['population'].apply(lambda x: int(x.split('.')[0].replace(',','')))

In [139]:
fig = px.scatter(stats_c, x="population_log10", y="mean_log10", trendline="ols", hover_name='region_slug',
                 title='Mean by Population'
                )
fig.layout.template = 'plotly_white'
x = [4, 7]
y = [-3.5542 + (i * 1.4932)  for i in x]
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode="lines",
        line=go.scatter.Line(color="red"),
        showlegend=False)
)
fig.show()

In [140]:
stats_c['ratio'] = stats_c['mean_log10'] / stats_c['population_log10']
lower_bound = stats_c['ratio'].describe(percentiles=[0.1, 0.9])['10%']


In [142]:
stats_c['ratio'].describe(percentiles=[0.1, 0.9])

count    84.000000
mean      1.065483
std       0.120745
min       0.659480
10%       0.854342
50%       1.114209
90%       1.161324
max       1.265622
Name: ratio, dtype: float64

In [141]:
lower_bound

0.854342099097369

In [114]:
fig.write_html(open(OUTPUT_PATH / 'mean_population.html', 'w'))

In [96]:
results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.462
Method:                 Least Squares   F-statistic:                     72.28
Date:                Fri, 03 Apr 2020   Prob (F-statistic):           7.05e-13
Time:                        12:38:31   Log-Likelihood:                -86.790
No. Observations:                  84   AIC:                             177.6
Df Residuals:                      82   BIC:                             182.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.5542      1.058     -2.415      0.018      -4.658      -0.450
x1             1.4932      0.176      8.502      0.000       1.144       1.843
==============================================================================
Omnibus:                       31.267   Durbin-Watson:                   1.873
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.616
Skew:                          -1.455   Prob(JB):                     5.08e-13
Kurtosis:                       5.777   Cond. No.                         87.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

count    84.000000
mean      1.065483
std       0.120745
min       0.659480
10%       0.854342
50%       1.114209
90%       1.161324
max       1.265622
Name: ratio, dtype: float64

In [119]:
stats_c['mean_log10'] = stats_c['mean'].apply(np.log10)
# stats_c['population_log10'] = stats_c['population'].apply(lambda x: np.log10(int(x.split('.')[0].replace(',',''))))
stats_c.set_index('population')['mean'].iplot(
    mode='markers', size=7, theme='custom', title='Mean by Population',
                          xTitle='Population (log10)', yTitle='Mean (log10 meter)')

In [80]:
stats_c['mean_log10'] = stats_c['mean'].apply(np.log10)
stats_c.set_index('mean_log10')['std_log10'].iplot(
    mode='markers', size=7, theme='custom', title='STD by Mean',
                          xTitle='Mean (log10)', yTitle='STD (log10 meter)')

In [82]:
stats_c['50%_log10'] = stats_c['50%'].apply(np.log10)
stats_c.set_index('mean_log10')['50%_log10'].iplot(
    mode='markers', size=7, theme='custom', title='Median by Mean',
                          xTitle='Mean (log10)', yTitle='Median (log10 meter)')

In [186]:
(stats_c['std'] / stats_c['mean']).iplot(kind='hist', bins=20)

In [182]:
stats_c['coef_var'] = (stats_c['std'] / stats_c['mean'])

In [191]:
stats_c['coef_var'].describe(percentiles=[0.1, 0.75, 0.9])

count    84.000000
mean      0.496310
std       0.516542
min       0.243161
10%       0.318471
50%       0.390516
75%       0.510733
90%       0.656462
max       5.000581
Name: coef_var, dtype: float64

In [194]:
stats_c[stats_c['coef_var'] > 0.51][['region_slug',  'coef_var']].sort_values(by='coef_var')

,region_slug,coef_var
36,joinville,0.521256
79,tijuana,0.531809
29,florianopolis,0.532589
53,paramaribo,0.579580
57,portofspain,0.581513
64,sanfernado,0.581871
50,nassau,0.597103
17,cordoba,0.606039
55,portauprince,0.618755
3,barcelona,0.630572


In [193]:
len(stats_c[stats_c['coef_var'] > 0.51])

21

In [153]:
pd.DataFrame() == df

ValueError: Can only compare identically-labeled DataFrame objects

In [154]:
type(df)

pandas.core.frame.DataFrame

In [169]:
import requests
import json
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json][maxsize:1000000000];
  way["highway"](-46.52605120190802,-23.5775555726401,-46.478157677738096,-23.619556225293707);
make stat number=count(ways),length=sum(length());  
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})

In [177]:
pd.DataFrame([response.json()['elements'][0]['tags']])

,length,number
0,2793022.061,2269


In [168]:
pd.read_

FileNotFoundError: File b'number\tlength\n2269\t2793022.061\n' does not exist

In [196]:
from shapely import wkt

In [197]:
url = 'http://docs.google.com/spreadsheets/d/197fccIfjwlsT-oeisHHLn2SuOvbtJjC0inoNuxfqrwI/export?format=csv&id'

In [199]:
df = pd.read_csv(url)

In [209]:
for i, p in enumerate(df1['region_shapefile_wkt'].tolist()):
    print(i)
    query = f"select ST_POLYGON('{p}')"
    pd.read_sql_query(query, conn)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111


In [226]:
df1 = pd.read_sql_query('select * from spd_sdv_waze_corona.dev_metadata_regions_metadata', conn)

In [228]:
df1

,country_name,country_iso,region_slug,region_name,region_type,population,dashboard,timezone,region_shapefile_wkt
0,Argentina,AR,cordoba,Córdoba,city,1422824,TRUE,America/Argentina/Cordoba,Polygon ((-64.30875060953628974 -31.2754180085...
1,Brazil,BR,porto_alergre,Porto Alegre,city,3385942,TRUE,,Polygon ((-51.11708480747174121 -30.1420845977...
2,Colombia,CO,medellin,Medellín,city,3550671,TRUE,America/Bogota,Polygon ((-75.54208320661969367 6.382917897523...
3,Ecuador,EC,quito,Quito,city,2255262,TRUE,America/Guayaquil,Polygon ((-78.45874968212353906 -0.00041587190...
4,Nicaragua,NI,managua,Managua,city,941824,TRUE,America/Managua,Polygon ((-86.35874916434539728 12.17458495986...
5,Uruguay,UY,montevideo,Montevideo,city,1363552,TRUE,America/Montevideo,Polygon ((-56.23375113878419995 -34.7587515798...
6,Argentina,AR,mendoza,Mendoza,city,808809,TRUE,America/Argentina/Mendoza,Polygon ((-68.8337503129608308 -32.80875144662...
7,Argentina,AR,rosario,Rosario,city,1018655,TRUE,America/Argentina/Cordoba,Polygon ((-60.71708417827297666 -32.8254181144...
8,Argentina,AR,buenosaires,Buenos Aires,city,14338718,TRUE,America/Argentina/Buenos_Aires,Polygon ((-58.75041764050466497 -34.3670848864...
9,Barbados,BB,bridgetown,Bridgetown,city,148952,TRUE,America/Barbados,Polygon ((-59.56666666669877941 13.10833333330...


In [227]:
for i, p in enumerate(df1['region_shapefile_wkt']):
    print(i)
    try:
        wkt.loads(p)
    except:
        print('error')

ERROR|%|s


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
error
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115


In [232]:
url = 'https://outlook.office.com/webhook/ec43a1f0-c349-4114-916e-5e394da76e24@9dfb1a05-5f1d-449a-8960-62abcb479e7d/IncomingWebhook/650e4a8f0ea34516b352b8fced6de36d/6f083b85-a41e-409c-b0f9-a92365fc3fa4'

In [245]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2019, 1, 1)
end_date = date(2019, 12, 31)
dates = []
for single_date in daterange(start_date, end_date):
    dates.append({
        'year': single_date.year,
        'month': single_date.month,
        'day': single_date.day,
        'dow': single_date.weekday() + 1,
        'sum_length': 0
    })
    
pd.DataFrame(dates)

,day,dow,month,sum_length,year
0,1,2,1,0,2019
1,2,3,1,0,2019
2,3,4,1,0,2019
3,4,5,1,0,2019
4,5,6,1,0,2019
5,6,7,1,0,2019
6,7,1,1,0,2019
7,8,2,1,0,2019
8,9,3,1,0,2019
9,10,4,1,0,2019


In [268]:
if not(False and True):
    print('oi')
else:
    print('false')

oi


In [276]:
pd.read_csv('https://docs.google.com/spreadsheets/d/1wvg1KFWZp4WhYVI4Gw_82bL_je_2WZHNLCcSnx95MTI/gviz/tq?tqx=out:csv&sheet=Fontes')

,Dado,Fonte,Data de Coleta
0,Projeção de população,IBGE,20 de março de 2020
1,Leitos por município,DATASUS CNES,20 de março de 2020
2,Ventiladores por município,DATASUS CNES,20 de março de 2020
3,Casos e mortes confirmados por município,Brasil.io,Diariamente


In [270]:
pd.read_csv('https://docs.google.com/spreadsheets/d/1wvg1KFWZp4WhYVI4Gw_82bL_je_2WZHNLCcSnx95MTI/export?format=csv&id')

,Descrição,U.M.,Valor,Data de Atualização,Fonte,Identificador
0,Taxa de reprodução básica no Cenário 1,NaN,3.74 - 3.97,03/21/2020,Wang et al. (2020),simulator_scenarios_nothing
1,Taxa de reprodução básica no Cenário 2,NaN,1.21 - 1.31,03/21/2020,Wang et al. (2020),simulator_scenarios_isolation
2,Taxa de reprodução básica no Cenário 3,NaN,0.28 - 0.37,03/21/2020,Wang et al. (2020),simulator_scenarios_lockdown
3,Tempo de resposta médio de teste para Covid19,dias,4 - 7,NaN,NaN,simulator_scenarios_test_delay
4,Percentual de casos diagnosticados que são gra...,percentual,12%,03/21/2020,CDC(2020),br_seir_parameters_i2_percentage
5,Percentual de casos diagnosticados que são crí...,percentual,2.5%,03/21/2020,CDC(2020),br_seir_parameters_i3_percentage
6,Taxa de mortalidade de todos os indivíduos sin...,percentual,2%,NaN,NaN,br_seir_parameters_fatality_ratio
7,Tempo em que pessoa com sintomas leves ficam d...,dias,6,NaN,NaN,br_seir_parameters_mild_duration
8,Tempo em que pessoa em estado grave permanece ...,dias,6,03/21/2020,Ferguson et al.(2020),br_seir_parameters_severe_duration
9,Tempo que pessoa em estado crítico permanece i...,dias,8,03/21/2020,Ferguson et al.(2020),br_seir_parameters_critical_duration
